# Основы геометрической оптики

В модели "геометрической оптики" предполагается, что распространение волн происходит вдоль **лучей**. В однородной среде лучи являются *прямыми*, а при возникновении неоднородностей показателя преломления $n$, направление распространения лучей *преломляются* по закону
$$
n_1 \sin \varphi_1 = n_2 \sin \varphi_2,
$$
где $\varphi_1$ --- угол падения из среды 1, $\varphi_2$ --- угол преломления в среде 2 (углы измеряются относительно нормали к поверхности раздела).

## Геометрическая оптика параксиальных систем

Далее будем рассматривать осесимметричные оптические системы, в которых лучи света распространяются под малыми углами к *оптической оси*. Для малых углов $\varphi\ll 1$ закон преломления имеет вид
$$
n_1 \varphi_{1} = n_2 \varphi_{2}
$$

Если луч составляет угол $\alpha \ll 1$ с оптической осью, считаем выполнеными равенства
$$
\mathrm{tg\,} \alpha \approx \sin \alpha \approx \alpha.
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import math 

def plotsection(A, B, **args):
    plt.plot([A[0], B[0]], [A[1], B[1]], **args)
def plotdot(A, **args):
    plt.plot([A[0]], [A[1]], linestyle='', **args)

class Ray:
    def __init__(self, x, y, slope):
        self.x = x
        self.y = y
        self.slope = slope    

def point(ray, s=0):
    return (ray.x + s, 
            ray.y + s * ray.slope)
    
class Cell:
    def __init__(self, x, R, n, refraction_func):
        self.x = x
        self.R = R
        self.n = n
        self.refraction_func = lambda ray: refraction_func(ray, self)

    def Fi(self):
        return self.n * self.R / (self.n - 1)
    
    def Fb(self):
        return self.R / (self.n - 1)

    def image(self, X0, Y0):
        Fi = self.Fi()
        Fb = self.Fb()
        X0 -= self.x
        # Положение изображения (по формуле Ньютона)
        Xs = Fi * Fb / (-Fb - X0) + Fi 
        Ys = - Y0 * (Xs / (-X0)) * (Fb / Fi)
        return (Xs + self.x, Ys)
    
    def plot(self, **args):
        plt.gca().add_patch(Arc((self.R + self.x, 0), 2*self.R, 2*self.R, 
                                angle=90, theta1=0, theta2=180,
                                **args))

# точное решение задачи о пересечении луча и сферической поверхности    
def cell_precise(ray, cell):    
    result = Ray(0., 0., 0.)
    alpha = math.atan(ray.slope)

    if abs(ray.slope) < 1e-9: # луч параллелен опт. оси    
        psi = math.asin(ray.y / cell.R)
        result.y = ray.y
    else:
        Q = ray.x - ray.y / ray.slope # точка пересечения с оптической осью
        psi = math.asin((cell.R + (cell.x - Q)) / cell.R * math.sin(alpha)) - alpha
        result.y = cell.R * math.sin(psi)
    result.x = cell.x + cell.R * (1 - math.cos(psi))
    
    # преломление
    result.slope = math.tan(math.asin(math.sin(psi + alpha) / cell.n) - psi)
    return [ray, result]


# точное решение задачи о пересечении луча и плоскости
def cell_simple(ray, cell):    
    result = Ray(cell.x, 0., 0.)
    alpha = math.atan(ray.slope)    
    result.y = ray.y + (cell.x - ray.x) * ray.slope
    psi = math.atan(result.y / cell.R)
    result.slope = math.tan(math.asin(math.sin(psi + alpha) / cell.n) - psi)
    return [ray, result]


# приближенное решение задачи о пересечении луча и плоскости  (параксиальное приближение)
def cell_paraxial(ray, cell):
    result = Ray(cell.x, 0., 0.)
    alpha = ray.slope    
    result.y = ray.y + (cell.x - ray.x) * alpha
    psi = result.y / cell.R    
    result.slope = -(psi - ((psi + alpha) / cell.n))
    return [ray, result]

In [ ]:
def plot_rays(X0, Y0, refractor, ALPHA=(-2.5, 2.5),
                   XLIM = (-10,10), YLIM = (-5,5), NRAY=11,                    
                   traceback=False, traceforward=False,
                   focals=True,
                   image=True,
                   title=''):
           
    plt.figure(figsize=(10,4))
    plt.xlim(*XLIM)
    plt.ylim(*YLIM)
    plt.title(title)    
    
    # оптическая ось
    plt.axline((0., 0.), slope=0., ls='-.', color='k',lw=1)

    # фокальные плоскости
    if focals:
        Fi = refractor.Fi() # фокусное расстояние пространства изображений
        Fb = refractor.Fb() # фокусное расстояние пространства предметов
        plt.axvline(Fi, color='r', ls='--', lw=1)
        plt.axvline(-Fb, color='r', ls='--', lw=1)
        plt.text(Fi+0.3, -0.7, f'$F\'={Fi:.1f}$', fontsize='large')    
        plt.text(-Fb+0.3, -0.7, f'$F={Fb:.1f}$', fontsize='large')

    refractor.plot(color='k', lw=2)

    # наклоны лучей
    if isinstance(ALPHA, (int, float)):
        ALPHA = (-ALPHA, ALPHA)         
    if isinstance(ALPHA, tuple):        
        slopes = np.tan(np.linspace(ALPHA[0] * np.pi / 180, ALPHA[1] * np.pi / 180, NRAY))    
    else:
        slopes = np.tan(np.array(ALPHA) * np.pi / 180)
        
    # построение пучка лучей
    for alpha in slopes:
        ray0 = Ray(X0, Y0, alpha)
        rays = refractor.refraction_func(ray0)
        
        N = len(rays)        
        if traceforward: # продолжения лучей предмета
            plotsection(point(rays[0], 0), point(rays[1], 0), color='r', linestyle='--', lw=0.5)
            plotsection(point(rays[0], -(-XLIM[0] + rays[0].x)), point(rays[0], -rays[0].x), color='b', linestyle='-', lw=0.5)
        else:
            plotsection(point(rays[0], 0), point(rays[1], 0), color='b', lw=0.5)
        for i in range(1, N - 1):
            plotsection(point(rays[i], 0), point(rays[i + 1], 0), color='b', lw=0.5)
        plotsection(point(rays[-1], 0), point(rays[-1], XLIM[1]), color='b', lw=0.5)
        if traceback: # продолжения лучей в изображения
            plotsection(point(rays[-1], 0), point(rays[-1], -rays[-1].x + XLIM[0]), color='r', linestyle='--', lw=0.5)        
            

    plotdot((X0, Y0), color='b', marker='*', label='Предмет')
    # расчётное положение изображения 
    if image:
        Xs, Ys = refractor.image(X0, Y0)    
        plotdot((Xs, Ys), color='r', marker='x', ms=8, label='Изображение (теория)')
    
    plt.legend()    

### Фокусирующие свойства кривых поверхностей

Основой для работы всех оптических приборов является *фоксирующее* свойство криволинейной поверхности.

Рассмотрим систему, разделённую на две части сферической поверхностью с радиусом кривизны $R$, пересекающей оптическую ось $x$ в точке $x=0$ (считаем $R>0$, если поверхность ориентирована выпуклостью против оси $x$). 

Пусть показатель преломления среды слева $n_1$, справа $n_2$. Определим *относительный показатель преломления* ячейки: $n=n_2/n_1$.

#### Фокусы элементарной ячейки --- луч, параллельный оси

Пустим слева направо луч, проходящий на расстоянии $y_1$ от оптической оси ($y_1\ll R$).

In [ ]:
plt.figure(figsize=(6,3),dpi=150)
plt.xlim(-4,16)
plt.ylim(-2,4)

# оптическая ось
plt.axline((0., 0.), slope=0., ls='-.', color='k',lw=1)

plt.gca().add_patch(Arc((5.0, 0), 10.0, 10.0, 
                                angle=90, theta1=0, theta2=180))
plotsection((-10,2), (0.4,2), color='b')
plotsection((.4,2), (5,0), color='k', ls='--')
plotsection((.4,2), (15,0), color='b')
plotsection((.4,2), (15,0), color='b')
plotsection((.4,2), (15,2), color='r', ls='--')
plotsection((.4,2), (-4,4), color='k', ls='--')
plt.text(3.3, 0.2, '$\\psi$')
plt.text(-1.2, 2.2, '$\\varphi_1$')
plt.text(2.5, 1.3, '$\\varphi_2$')
plt.text(4.2, 1.6, '$\\alpha_2$')
plt.text(-3.8, 1.7, '$y$')
plt.text(1.7, 0.9, '$R$')
plt.text(5.0, -0.4, '$R$')
plt.text(15.0, -0.4, '$f_2$')
plt.show()

На рисунке отмечены углы: угол падения $\varphi_1 = \psi \approx y_1/ R$, и по закону преломления $n\varphi_2 \approx \varphi_1$, откуда $\varphi_2 \approx \psi/n$. 

Угол преломлённого луча относительно оптической оси:
$$
\alpha_2 = \psi - \varphi_2 = \frac{n-1}{n} \psi
$$

Выразим расстояние, на котором преломленный луч пересечёт оптическую ось:
$$
\boxed{f_2 = \frac{y}{\mathrm{tg\,} \alpha_2} \approx \frac{n_2 R}{n_2-n_1} = \frac{n R}{n-1}.}
$$
Видим, что расстояние $f_2$ *не зависит от $y$*, то есть *параллельный пучок* лучей с любыми $y_1$ соберётся в одной точке $F_2$. Расстояние $f_2$ называют *передним фокусным расстоянием* оптической ячейки.

Пустим такой же пучок лучей, параллельных отпической оси, но в обратном направлении. Они соберутся в точке $F_1$, находящейся от точки $O$ на расстоянии 
$$
\boxed{f_1 = \frac{n_1 R}{n_2-n_1} = \frac{R}{n-1}.}
$$
Это расстояние называют *задним фокусным расстоянием* оптической ячейки.

Заметим, что для применимости выводов выше (и ниже) важно, чтобы луч проходил вблизи оптической оси, то есть должен быть мал угол $\psi \ll 1$.

In [ ]:
plot_rays(-1000.0, 0.0, 
          Cell(0., R=5.0, n=1.5, refraction_func=cell_paraxial),
          ALPHA=0.1, NRAY=10, XLIM =(-18,40), YLIM=(-3,3),
          traceback=False,          
          title='Фокусировка параллельного пучка из пространства предметов')
plt.show()
plot_rays(-10.0, 0.0, 
          Cell(0., R=5.0, n=1.5, refraction_func=cell_paraxial),
          ALPHA=10., NRAY=10, XLIM =(-18,40), YLIM=(-3,3),
          traceback=False,  image=False,        
          title='Фокусировка параллельного пучка из пространства изображений')
plt.show()

#### Фокальные плоскости --- наклонный луч

Рассмотрим параллельный пучок лучей, идущих под углом $\alpha_1$ к оптической оси (сперва -- слева направо).

Пусть некоторый луч пересекает границу ячейки на высоте $y_1$, $\psi\approx y_1/R$. Повторяя построение, аналогично 
находим: $\varphi_1 = \psi + \alpha_1$, $\varphi_2 = \varphi_1 /n$, и
$$
\alpha_2 = \psi - \varphi_2 = \frac{n-1}{n}\psi - \frac{\alpha_1}{n} = \frac{y_1}{f_2} - \frac{\alpha_1}{n}
$$
Замечаем, что смещение луча по оси $y$ на расстоянии $f_2$, равное
$
\Delta y = y_1 - \alpha_2 f_2 = \frac{\alpha_1}{n},
$
не зависит от $y_1$. Следовательно, все лучи пучка соберутся
в одной точке с координатами $(x_2, y_2)$,
где 
$$
x_2 = f_2
$$
и
$$
y_2 \approx \alpha_1 f_2 / n.
$$

Все возможные такие точки пересечения при любых $\alpha_1$ образую *фокальную плоскость пространства изображений*.

Если рассмотреть пучки, идущие в обратную сторону, они будут фокусироваться в *фокальной плоскости пространства предметов*, находящейся на расстоянии $f_1$ от начала отсчёта.

In [ ]:
plot_rays(-1000.0, -100.0, 
          Cell(0., R=5.0, n=1.5, refraction_func=cell_paraxial),
          ALPHA=(5.65,5.81), NRAY=10, XLIM =(-18,40), YLIM=(-3,3),
          traceback=False,          
          title='Фокусировка наклонного пучка из пространства предметов')
plt.show()
plot_rays(-10.0, 1.0, 
          Cell(0., R=5.0, n=1.5, refraction_func=cell_paraxial),
          ALPHA=10., NRAY=10, XLIM =(-18,40), YLIM=(-3,3),
          traceback=False,  image=False,        
          title='Фокусировка наклонного пучка из пространства изображений')
plt.show()

#### Изображение предмета в оптической ячейке

Рассмотрим светяющуюся точку $S_1$ с координатами $(x_1,y_1)$, находящуюся слева от границы ($x_1 < 0$). Точку $S_1$ будем называть *предметом*.

Можно доказать (например, прямым построением) следующие два важнейших утверждения, справедливые для параксиальных лучей (и только для них!):
1) Пучок параксиальных лучей, выходящих из $S_1$, соберётся в некоторой точке $S_2$, которую называют *изображением*.
2) Множеству точек $S_1$, лежащих в некоторой плоскости, *перпендикулярной* оптической оси, будут соответствовать изображения $S_2$, также находящиеся в одной плоскости, перпендикулярной оптической оси.


Если точка $S_2$ лежит в области $x>0$, то есть является *реальной* точкой персечения лучей, то такое изображение называют *действительным*. Действительное изображение можно увидеть, двумя способами: 1) если поставить в месте пересечения лучей непрозрачный *экран*, 2) если расположить глаз за точкой $S_2$ (так, чтобы в глаз попадали проложения расходящихся из точки $S_2$ лучей).

Если точка $S_2$ лежит в области $x<0$, то есть пересекаются *продолжения* лучей, то такое изображение называют *мнимым*. Мнимое изображение можно увидеть непосредственно глазом.



In [ ]:
plot_rays(-17.0, 0.25, 
          Cell(0., 5.0, 1.5, refraction_func=cell_precise), 
          ALPHA=2, NRAY=11, XLIM =(-18,40), YLIM=(-2.5,2.5),          
          title='Действительное изображение точки в параксиальных лучах')
plt.show()
plot_rays(-5, 0.5, 
          Cell(0., 5.0, 1.5, refraction_func=cell_precise),
          ALPHA=5, NRAY=11, XLIM =(-18,40), YLIM=(-2.5,2.5), 
          traceback=True,          
          title='Мнимое изображение точки в параксиальных лучах')
plt.show()

Возможно также рассмотрение ситуации, когда *предмет* является *мнимым*: это означает, что на границу слева падают лучи, продолжения которых сходятся в некоторой точке с координатой $x>0$.

In [ ]:
plot_rays(14, 0.5, 
          Cell(0., 5.0, 1.5, refraction_func=cell_precise),
          ALPHA=2, NRAY=11, XLIM =(-18,40), YLIM=(-2.5,2.5), 
          traceback=False, traceforward=True,         
          title='Действительное изображение мнимого предмета')
plt.show()

Если лучи сильно не параксиальны или предмет расположен слишком далеко от оптической оси, продолжения лучей **не будут** сходиться в одну точку.

Такие отклонения реальных лучей от законов параксиальной оптики называют **геометрическими аберрациями**.

In [ ]:
plot_rays(-17.0,1.0, Cell(0., 5.0, 1.5, refraction_func=cell_precise),
          ALPHA=10, NRAY=31, XLIM =(-18,40), YLIM=(-5,5),
          traceback=False,          
          title='Отсутствие чёткого изображения точки в НЕ параксиальных лучах')
plt.show()

### Формулы для нахождения положения изображения

Поскольку все параксиальные лучи, исходящие из предмета, собираются в одной точке, для нахождения положения изображения достаточно рассмотреть ход только *двух* лучей. Наиболее удобны для рассмотрения любые два из следующих трёх лучей:
* луч, параллельный оптической оси --- его продолжение пройдет через фокус пространства изображений
* луч, проходящий через фокус пространства предметов --- он выйдет параллельно оптической оси
* луч, проходящий через начало координат --- он преломится на угол $\alpha_2 = \alpha_1 / n$

In [ ]:
plot_rays(-17.0, 0.5, 
          Cell(0., 5.0, 1.5, refraction_func=cell_paraxial), 
          ALPHA=[0, -1.7, -4], XLIM =(-18,40), YLIM=(-1.5,1.5),          
          title='Действительное изображение точки в параксиальных лучах')
plt.text(0.5,0.55,'$y_1$')
plt.text(0.5,-0.85,'$y_2$')
plt.text(-17.5,-0.2,'$-x_1$')
plt.text(36,-0.2,'$x_2$')
plt.text(5,-1.2,'$n$')
plt.show()

Пусть координаты предмета $S_1(-x_1, y_1)$, изображения $S_2(x_2, y_2)$ (для удобства считаем, что если предмет действительный, то $x_1>0$). Фокусные расстояния ячейки равны $f_1$ и $f_2$,
координаты фокусов: $F_1(-f_1,0)$, $F_2(f_2,0)$.

Из закона преломления центрального луча получим *формулу увеличения*:
$$
\boxed{\Gamma \equiv \frac{y_2}{y_1} = -\frac{1}{n}\frac{x_2}{x_1}}.
$$
Она показывет, во сколько раз каждая точка удаляется от оптической оси.
Отрицательное значение $\Gamma$ означает, что изображение *перевёрнутое*.

Из подобия треугольников также имеем
$$
\frac{y_1}{f_2} = \frac{-y_2}{x_2 - f_2}\qquad \text{и} \qquad
\frac{-y_2}{f_1} = \frac{y_1}{x_1-f_1}.
$$
Исключая $y_1/y_2$, найдём:
$$
\boxed{(x_1 - f_1) (x_2 - f_2) = f_1 f_2}.
$$
Эту формулу назвают *формулой Ньютона*. 

Раскроем в формуле Ньютона скобки (тогда сократится слагаемое $f_1f_2$) и поделим результат на $x_1x_2$. Получим ещё одну формулу, которую удобно использовать при расчётах:
$$
\boxed{\frac{f_1}{x_1}+\frac{f_2}{x_2}=1}.
$$

## Тонкая линза

Тонка линза --- это две элементарные ячейки, расположенные в одной точке.

Рассмотрим для примера двояковыпуклую линзу. Путь радиусы кривизны поверхностей линз равны $R_1>0$ и $R_2>0$, а показатель преломления материала линзы $n$ (показатель преломления окружающего воздуха 1). Тогда имеем две последовательные элементарные ячейки: первая с радиусом кривизны $R_1$ и относительным показателем преломления $n$, а вторая --- с радиусом кривизны $-R_2$ ("смотрит" в другую сторону) и обратным относительным показателем преломления $1/n$.

Пусть расстояние от линзы до предмета равно $a$, а от линзы до изображения $b$. Тогда с учётом формул для фокусных расстояний на первой поверхности имеем
$$
\frac{R_1/(n-1)}{a}+\frac{nR_1/(n-1)}{x^{\star}}=1,
$$
где $x^{\star}$ --- положение промежуточного изображения "внутри" линзы. А на второй:
$$
\frac{-nR_2/(n-1)}{-x^{\star}}+\frac{-R_2/(n-1)}{b}=1.
$$
Поделим первое на $R_1$, второе на $R_2$ и вычтем одно из другого. Получим известную *формулу тонкой линзы*:
$$
\boxed{\frac{1}{a} + \frac{1}{b} = \frac{1}{f},}
$$
где мы ввели обозначение для фокусного расстояние тонкой линзы:
$$
\frac{1}{f} = (n-1)\left(\frac{1}{R_1} + \frac{1}{R_2} \right).
$$
Формула Ньютона для линзы:
$$
(a-f)(b-f)=f^2.
$$
Линейное увеличение линзы:
$$
\boxed{\Gamma = -\frac{b}{a} = -\frac{b - f}{f} = -\frac{f}{a - f}}.
$$

Если $f>0$, линза называется *положительной* или *собирающей*. Если $f<0$ --- *отрицательной* или *рассеивающей*.
* Если предмет находится дальше фокуса $a>f$, собирающая линза даёт *действительное* ($b>0$) и *перевёрнутое* ($y_2<0$) изображение. При $f<a<2f$ изображение *увеличено* ($|y_2/y_1|>1$), при $a>2f$ --- *уменьшено* ($|y_2/y_1|<1$).
* Если предмет находится ближе фокуса $a<f$, собирающая линза даёт *мнимое* ($b<0$), *прямое* ($y_2>0$) и *увеличенное* изображение.
* Рассеивающая линза *всегда* даёт *мнимое*, *прямое* и *уменьшенное* изображение.

### Лупа
Положительная линза, через которую рассматривают глазом мнимое увеличенное изображение предмета, называется *лупой*. Для максимального увеличения предмет нужно помещать чуть ближе точки фокуса лупы ($a<f$).

Глазом удобно рассматривать предметы, находящиеся на *расстоянии наилучшего зрения* $L_{зр}$ (обычно считается, что при нормальной остроте зрения $L=25$ см). Если расположить глаз вплотную к лупе, то наиболее удобное положение изображения будет равно $b=-L$. Поэтому увеличение лупы равно
$$
\Gamma_{л} = \frac{L_{зр}}{a} = \frac{L_{зр}}{f} + 1.
$$

#### Аккомодация глаза на бесконечность, угловое увеличение
Разместим предмет точно в фокус лупы. Тогда формально и расстояние до изображения и его линейные размеры окажутся *бесконечными*. Однако конечным останется их *отношение*.
Пусть $\alpha = y / x$ --- угол, под которым виден предмет или его изображение из центра линзы.
Определим *угловое увеличение* как линзы
$$
\gamma = \frac{\alpha_2}{\alpha_1}.
$$

Для лупы имеем: если рассматривать предмет на расстоянии наилучешго зрения от глаза, то его угловой размер $\alpha_1 = y_1/L$. Поместим между предметом и глазом лупу так, чтобы предмет оказался в её фокусе лупы ($a=f$). Тогда угловой размер изображения будет равен $\alpha_2 = y_1/f$, и, следовательно увеличение лупы:
$$
\gamma_л = \frac{L_{зр}}{f}.
$$

In [ ]:
class Lens:
    def __init__(self, X0, R1, R2, n, H, cell_func=cell_precise, D=1.0):                
        self.first = Cell(X0 - H/2, R1, n, cell_func)
        self.second = Cell(X0 + H/2, R2, 1/n, cell_func)        
        self.cell_func = cell_func
        self.x = X0
        self.H = H
        self.D = D

    def Fi(self):
        return 1 / ((self.first.n - 1) * ( 1 / self.first.R - 1 / self.second.R))
    
    def Fb(self):
        return self.Fi()
        
    def H(self):
        return self.second.x - self.first.x

    def plot(self, **args):
        if self.H == 0:
            plt.arrow(self.x, 0, 0, self.D/2, width=0.1)
            plt.arrow(self.x, 0, 0, -self.D/2, width=0.1)
        else:            
            self.first.plot(**args)
            self.second.plot(**args)            

    def image(self, X0, Y0):        
        return self.second.image(*self.first.image(X0, Y0))
        
    def refraction_func(self, ray):
        ray1 = self.cell_func(ray, self.first)[1]
        result = self.cell_func(ray1, self.second)[1]
        return [ray, ray1, result]

In [ ]:
plot_rays(-7.5, 0.1, Lens(0., 5, -5, 1.5, 0.5, cell_precise),
          ALPHA=2, NRAY=11, XLIM =(-10,20), YLIM=(-1.,1.),          
          title='Действительное увеличенное изображение в собирающей тонкой линзе')
plt.show()
plot_rays(-3.5, 0.1, Lens(0., 5, -5, 1.5, 0.5, cell_precise),
          ALPHA=2, NRAY=11, XLIM =(-15,10), YLIM=(-1.,1.),
          traceback=True,
          title='Мнимое увеличенное изображение в собирающей тонкой линзе')
plt.show()
plot_rays(-6.0, 0.25, Lens(0., -5, 5, 1.5, 0.5, cell_precise),
          ALPHA=2, NRAY=11, XLIM =(-15,10), YLIM=(-1.,1.),
          traceback=True,
          title='Мнимое уменьшенное изображение в рассеивающей тонкой линзе')
plt.show()

## Микроскоп

Микроскоп --- это *две* линзы (объектив и окуляр), расположенные на некотором расстоянии друг от друга.

Чтобы микроскоп давал увеличенное изображение, объектив должен быть собирающей линзой. Окуляр может быть как рассивающим, так и собирающим.

Пусть $L$ --- растояние между объективом и окуляром. Обычно $L\gg f_{об},f_{ок}$. 

Изучаемый предмет помещают вблизи фокуса объектива, чуть дальше от него ($a > f_{об}$). Расстояние $a$ подбирается таким образом, чтобы промежуточное изображение оказалось точно в фокальной плоскости окуляра ($b^{\star} = L - f_{ок}$ и $a^{\star} = f_{ок}$). Далее окуляр используется как *лупа* для рассматривания изображения.

Увеличение микроскопа есть *произведение* увеличений объектива и окуляра.

Тогда для объектива:
$$
\gamma_{об} = \frac{L-f_{ок}}{f_{об}} \approx \frac{L}{f_{об}}
$$
для окуляра:
$$
\gamma_{ок} = \frac{L_{зр}}{f_{ок}}
$$
Итого угловое увеличение микроскопа (с учётом сделанных приближений):
$$
\gamma_{микро} = \gamma_{об}\gamma_{ок} \approx \frac{L L_{зр}}{f_{об}f_{ок}}.
$$

Если сместить окуляр чуть ближе к объективу, увеличенное финальное изображение можно переместить на расстояние наилучшего зрения. Тогда нетрудно получить формулу линейного увеличения:
$$
\Gamma^{наил.зр.}_{микро} = \frac{L-f_{ок}}{f_{об}} \frac{L_{зр}-f_{ок}}{f_{ок}}
$$

Если, наоборот, немного отодвинуть окуляр, то финальное изображение станет *действительным* и его можно будет спроецировать на экран (получится *проекционный микроскоп*). Если $L_{эк}$ --- расстояние от окуляра до экрана, то можно найти
$$
\Gamma^{проекц.}_{микро} = \frac{L-f_{ок}}{f_{об}} \frac{L_{эк}+f_{ок}}{f_{ок}}
$$

In [ ]:
class Microscope:
    def __init__(self, L, f1, f2, H=0, cell_func=cell_precise, D=1.0):
        self.objective = Lens(0., f1, -f1, 1.5, H, cell_func, D)
        self.ocular = Lens(L, f2, -f2, 1.5, H, cell_func, D)              

    def plot(self, **args):
        self.objective.plot(**args)
        self.ocular.plot(**args)
        
    def image(self, X0, Y0):
        return self.ocular.image(*self.objective.image(X0, Y0))
        
    def refraction_func(self, ray):
        rays1 = self.objective.refraction_func(ray)
        rays2 = self.ocular.refraction_func(rays1[-1])
        return rays1 + rays2[1:]

TUBE = 11.8
M = Microscope(TUBE, 2.0, 2.0, 0.0, cell_paraxial, D=1.0)
plot_rays(-2.5, 0.01, M,
          ALPHA=10, NRAY=21, XLIM =(-7,20), YLIM=(-0.75,0.75),   
          traceback=True, focals=False,
          title='Изображение в микроскопе')
plt.axvline(M.objective.Fi(), color='r', ls='--', lw=1)
plt.axvline(-M.objective.Fi(), color='r', ls='--', lw=1)
plt.axvline(M.ocular.Fi() + TUBE, color='r', ls='--', lw=1)
plt.axvline(-M.ocular.Fi() + TUBE, color='r', ls='--', lw=1)
plt.show()

## Телескоп / подзорная труба

*Телескопической* называтеся оптическая система, которая параллельные пучки лучей преобразует в параллельные. Простейший телескоп также состоит из двух линз --- объектива и окуляра. При этом передний фокус объектива должен *совпадать* с задним фокусом окуляра (расстояние между линзами $L=f_{об}+f_{ок}$).

Телескоп (поздорная труба) предназначена для рассматривания удалённых объектов ($a_{об}\gg f_{об}$). Поэтому промежуточное изображение будет находиться практически в фокальной плоскости объектива ($b_{об} \approx f_{об}$). Это изображение рассматривается в окуляр как в лупу ($a_{ок} \approx f_{ок}$).

Как видно из рисунка ниже, *угловое увеличение* подзорной трубы, настроенной на бесконечность:
$$
\gamma_{теле} =  \frac{f_{об}}{f_{ок}} .
$$
Подзорная труба увеличивает, если $f_{об} > f_{ок}$ (и во столько же раз уменьшает, если смотреть через неё с обратной стороны).

Подзорную трубу с двумя собирающими линзами называют *трубой Кеплера*. Если в качестве окуляра использовать рассеивающую линзу (*труба Галилея*) с тем же по модулю фокусным расстоянием $-f_{ок}$, то при сохранении увеличения можно существенно уменьшить длину трубы ($L = f_{об} - f_{ок}$).

In [ ]:
TUBE = 4
M = Microscope(TUBE, 3.0, 1.0, 0, cell_paraxial, D=2.0)
plot_rays(-1000, -100, M,
          ALPHA=(5.65,5.77), NRAY=15, XLIM =(-7,10), YLIM=(-3,3),   
          traceback=True, focals=False,
          title='Изображение удалённого предмета в телескопе')
plt.axvline(M.objective.Fi(), color='r', ls='--', lw=1)
plt.axvline(-M.objective.Fi(), color='r', ls='--', lw=1)
plt.axvline(M.ocular.Fi() + TUBE, color='r', ls='--', lw=1)
plt.axvline(-M.ocular.Fi() + TUBE, color='r', ls='--', lw=1)
plt.text(1.5,-1, '$f_{об}$')
plt.text(3.2,-1, '$f_{ок}$')
plt.show()

Для наблюдения *не слишком* далёких объектов можно незначительно изменять расстояние между фокусами объектива и окуляра (раздвижные подзорные трубы).

Интересно, что *линейное* увеличение подзорной трубы меньше единицы (!). Применяя последовательно найденные выше формулы увеличения линз, можно получить:
$$
\Gamma_{теле} = \frac{L-f_{об}}{f_{об}} \frac{f_{ок}}{L-f_{об}} = \frac{f_{ок}}{f_{об}} = \frac{1}{\gamma_{теле}}
$$
То есть подзорная труба *уменьшает* предметы, приближая их.

In [ ]:
TUBE = 4
M = Microscope(TUBE, 3.0, 1.0, 0, cell_paraxial, D=2.0)
plot_rays(-100, -1, M,
          ALPHA=(0,1), NRAY=15, XLIM =(-100,20), YLIM=(-3,3),   
          traceback=True, focals=False,
          title='Уменьшенное и приближенное изображение в подзорной трубе')
plt.axvline(M.objective.Fi(), color='r', ls='--', lw=1)
plt.axvline(-M.objective.Fi(), color='r', ls='--', lw=1)
plt.axvline(M.ocular.Fi() + TUBE, color='r', ls='--', lw=1)
plt.axvline(-M.ocular.Fi() + TUBE, color='r', ls='--', lw=1)
plt.show()